In [1]:
import pandas as pd
import numpy as np
import joblib

#Visualización
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

#Métricas
import sklearn as sk
from sklearn import tree
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import confusion_matrix, classification_report, make_scorer

#Configuración Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

#Regressors
from xgboost import XGBRegressor

from keras import backend as K
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold

#Análisis de Sentimientos
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD

2024-11-12 13:00:03.458539: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-12 13:00:03.461827: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-12 13:00:03.472260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731427203.489837  266329 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731427203.494870  266329 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 13:00:03.512030: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
user_stories_train = pd.read_csv('data/train.csv', low_memory=False)

In [3]:
user_stories_train.head()

,id,title,description,project,storypoint
0,5660,Error enabling Appcelerator services during ap...,"When creating the default app, I encountered t...",project8,3
1,9014,Create a maintenance branch,"As a developer, I'd like to have a maintenance...",project6,5
2,4094,Service Activity Monitoring Backend integrated...,SAM API used by SAM GUI,project1,5
3,811,fs::enter(rootfs) does not work if 'rootfs' is...,I noticed this when I was testing the unified ...,project5,2
4,4459,transform processor with script option is broken,Creating the following stream throws exception...,project6,2


In [4]:
user_stories_train.dtypes

id              int64
title          object
description    object
project        object
storypoint      int64
dtype: object

In [5]:
# Vemos cantidad de filas y columnas
user_stories_train_size = user_stories_train.shape
print("Cantidad de columnas: %d" % user_stories_train_size[1])
print("Cantidad de filas: %d" % user_stories_train_size[0])

Cantidad de columnas: 5
Cantidad de filas: 7900


In [6]:
# Analizamos si hay filas duplicadas
user_stories_train_total = len(user_stories_train)
user_stories_train_unique = user_stories_train.drop_duplicates()
user_stories_train_unique_size = len(user_stories_train_unique)
print(f'Se eliminaron: {user_stories_train_total - user_stories_train_unique_size} filas duplicadas')

Se eliminaron: 0 filas duplicadas


In [7]:
# Vemos cantidad de datos faltantes
user_stories_train.isna().sum()

id             0
title          0
description    0
project        0
storypoint     0
dtype: int64

In [8]:
#Verifico balanceo de clases
user_stories_train['storypoint'].value_counts(normalize=True)*100

storypoint
3     23.392405
5     21.430380
1     20.620253
2     16.151899
8     12.974684
4      2.101266
13     1.949367
10     0.405063
20     0.392405
6      0.215190
16     0.088608
12     0.075949
40     0.063291
21     0.037975
15     0.025316
7      0.025316
32     0.012658
34     0.012658
14     0.012658
24     0.012658
Name: proportion, dtype: float64

### Preparamos el dataset

In [9]:
#Creamos un dataset con features a usar para clasificar
user_stories_x = user_stories_train['description'].copy()

#Creamos un dataset con la variable target 'storypoint'
user_stories_y = user_stories_train['storypoint'].copy()

#Genero los conjuntos de train y test
x_train, x_test, y_train, y_test = train_test_split(user_stories_x,
                                                   user_stories_y,
                                                   test_size=0.3,  #proporcion 70/30
                                                   random_state=2) #semilla

In [10]:
x_train

2646    The plugin will need to contribute the icon. W...
2724    The timer: component is used to generate messa...
5159    h5.Description:  In CLI we have the ability to...
976     We need to refactor the way how we package and...
2187    Standalone Admin currently has no shiny banner...
                              ...                        
3606    h5. Description:  When launching Studio with C...
5704    When a .tss file contains certain grammars, Ti...
6637    The data that is entering a broadcast stream c...
2575    Provide the infrastructure for HTTP GET /compl...
7336    Studio shows an Empty Preferences window when ...
Name: description, Length: 5530, dtype: object

In [11]:
y_train

2646    8
2724    2
5159    8
976     2
2187    3
       ..
3606    5
5704    2
6637    2
2575    3
7336    3
Name: storypoint, Length: 5530, dtype: int64

## XGBoost

###Cross Validation

In [12]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/users/pablo.prieto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/users/pablo.prieto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/users/pablo.prieto/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def lemmatizationAndStopwords(text):
    wordNetLemmatizer = WordNetLemmatizer()
    stopwordSets = set(stopwords.words('english'))
    words = word_tokenize(text)
    wordsFilter = [wordNetLemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stopwordSets and word.isalpha()]
    if not wordsFilter:
        return "empty"
    return ' '.join(wordsFilter)

In [ ]:
##KFOLD CV usando XGBRegressor (los mejores atributos, hiperparametros,etc)
from sklearn.pipeline import Pipeline

#The Best
# {'xgb_regressor__subsample': 0.7, 'xgb_regressor__reg_lambda': 0.1, 'xgb_regressor__reg_alpha': 0, 'xgb_regressor__random_state': 42, 
#  'xgb_regressor__objective': 'reg:squarederror', 'xgb_regressor__n_estimators': 500, 'xgb_regressor__max_depth': 10, 
#  'xgb_regressor__learning_rate': 0.3, 'xgb_regressor__gamma': 0.1, 'xgb_regressor__colsample_bytree': 1, 'tfidf__use_idf': False, 
#  'tfidf__sublinear_tf': True, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 1), 'tfidf__min_df': 5, 
#  'tfidf__max_features': 2000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word'} 

parameters = {
    "tfidf__analyzer": ["word"],
    # "tfidf__smooth_idf": [True,False],
    "tfidf__ngram_range": [(1, 1),(1, 2)],
    # "tfidf__ngram_range": [(1, 2)],
    "tfidf__use_idf": [True,False],
    "tfidf__stop_words": [['english']],
    "tfidf__max_features": [1000, 2000],
    "tfidf__min_df": [2,5],
    # 'tfidf__max_df': [0.75, 0.8, 1.0],
    "tfidf__max_df": [0.75, 0.8],
     # "tfidf__max_features": [2000],
    # "tfidf__min_df": [2],
    # 'tfidf__max_df': [0.8],
    "tfidf__sublinear_tf": [True,False],
    "svd__n_components": [100, 200, 300, 400, 500],
    "svd__random_state" :[42],
    'xgb_regressor__objective': ['reg:squarederror'],
    'xgb_regressor__random_state' :[42], 
    'xgb_regressor__n_estimators': [100,200,500],
    'xgb_regressor__max_depth': [10,20,30],
    'xgb_regressor__learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'xgb_regressor__subsample': [0.6, 0.7, 0.8, 0.9, 1],
    'xgb_regressor__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1],
    'xgb_regressor__gamma': [0, 0.1, 0.2, 0.3, 0.4, 0.5],
    'xgb_regressor__reg_alpha': [0, 0.01, 0.1, 1, 10],
    'xgb_regressor__reg_lambda': [0, 0.01, 0.1, 1, 10]
}

randomcv_best_score = None

#Iteración Cantidad de splits para el Cross Validation
# for folds in [9,10,12,15,18]:
for folds in [5,6,7,8,9,10]:
# for folds in [5,10]:
    
    #Kfold
    kfoldcv = KFold(n_splits=folds, shuffle=True, random_state=42)

    #Regressors
    pipeline = Pipeline(
        steps=[
            ("tfidf", TfidfVectorizer(preprocessor=lemmatizationAndStopwords)),
            # ("tfidf", TfidfVectorizer()),
            ('svd', TruncatedSVD()),
            ("xgb_regressor", XGBRegressor()),
        ]
    )
    
    #Metrica que quiero optimizar root_mean_squared_error
    scorer_fn = make_scorer(root_mean_squared_error)

    #Random Search Cross Validation
    #Cantidad de combinaciones que quiero probar
    n=10
    randomcv_it = RandomizedSearchCV(estimator = pipeline,
                                     param_distributions = parameters,
                                     scoring = scorer_fn,
                                     cv = kfoldcv,
                                     n_iter = n) 

    # #Grid Search Cross Validation
    # randomcv_it = GridSearchCV(estimator = pipeline,
    #                                  param_grid = parameters,
    #                                  scoring = scorer_fn,
    #                                  cv = kfoldcv) 

    #Busco los hiperparamtros que optimizan F1 Score
    randomcv_it.fit(x_train,y_train);
    
    #Mejores hiperparametros del arbol
    print("folds: ", folds)
    print(randomcv_it.best_params_)
    #Mejor métrica
    print("root_mean_squared_error: ", randomcv_it.best_score_)
    print(" ")
    print("mean_test_score: ", randomcv_it.cv_results_['mean_test_score'])
    
    #Nos quedamos con el menor error root_mean_squared_error
    if randomcv_best_score is None:
        randomcv_best_score = randomcv_it
    elif randomcv_it.best_score_ < randomcv_best_score.best_score_:
        randomcv_best_score = randomcv_it

folds:  5
{'xgb_regressor__subsample': 0.7, 'xgb_regressor__reg_lambda': 0.1, 'xgb_regressor__reg_alpha': 0, 'xgb_regressor__random_state': 42, 'xgb_regressor__objective': 'reg:squarederror', 'xgb_regressor__n_estimators': 500, 'xgb_regressor__max_depth': 10, 'xgb_regressor__learning_rate': 0.3, 'xgb_regressor__gamma': 0.1, 'xgb_regressor__colsample_bytree': 1, 'tfidf__use_idf': False, 'tfidf__sublinear_tf': True, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 1), 'tfidf__min_df': 5, 'tfidf__max_features': 2000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word'}
root_mean_squared_error:  3.0159159380511493
 
mean_test_score:  [2.92330621 2.85599097 2.85119092 2.82276505 2.97847267 2.8901417
 2.82367213 2.87125503 2.82065449 3.01591594]


folds:  6
{'xgb_regressor__subsample': 0.6, 'xgb_regressor__reg_lambda': 1, 'xgb_regressor__reg_alpha': 0.1, 'xgb_regressor__random_state': 42, 'xgb_regressor__objective': 'reg:squarederror', 'xgb_regressor__n_estimators': 500, 'xgb_regressor__max_depth': 30, 'xgb_regressor__learning_rate': 0.3, 'xgb_regressor__gamma': 0.2, 'xgb_regressor__colsample_bytree': 0.6, 'tfidf__use_idf': False, 'tfidf__sublinear_tf': False, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 2, 'tfidf__max_features': 1000, 'tfidf__max_df': 0.75, 'tfidf__analyzer': 'word'}
root_mean_squared_error:  3.0278597575327697
 
mean_test_score:  [2.84593805 2.78977691 2.84076132 2.89010363 3.02785976 2.97155373
 2.79617854 2.85241136 2.94934541 2.86745071]


folds:  7
{'xgb_regressor__subsample': 0.6, 'xgb_regressor__reg_lambda': 0.1, 'xgb_regressor__reg_alpha': 1, 'xgb_regressor__random_state': 42, 'xgb_regressor__objective': 'reg:squarederror', 'xgb_regressor__n_estimators': 500, 'xgb_regressor__max_depth': 30, 'xgb_regressor__learning_rate': 0.3, 'xgb_regressor__gamma': 0.4, 'xgb_regressor__colsample_bytree': 1, 'tfidf__use_idf': False, 'tfidf__sublinear_tf': False, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 1), 'tfidf__min_df': 2, 'tfidf__max_features': 1000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word'}
root_mean_squared_error:  3.05369929415108
 
mean_test_score:  [2.77242345 2.86406871 2.79105078 3.05369929 2.78438385 2.80885404
 2.78349135 2.83704796 2.77721711 2.78574801]

 
folds:  8
{'xgb_regressor__subsample': 0.7, 'xgb_regressor__reg_lambda': 0.1, 'xgb_regressor__reg_alpha': 0.1, 'xgb_regressor__random_state': 42, 'xgb_regressor__objective': 'reg:squarederror', 'xgb_regressor__n_estimators': 100, 'xgb_regressor__max_depth': 20, 'xgb_regressor__learning_rate': 0.3, 'xgb_regressor__gamma': 0.2, 'xgb_regressor__colsample_bytree': 1, 'tfidf__use_idf': False, 'tfidf__sublinear_tf': False, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 2, 'tfidf__max_features': 1000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word'}
root_mean_squared_error:  3.0248804916226493
 
mean_test_score:  [2.86963099 2.76547353 2.80707325 2.90352536 3.02488049 2.84326477
 2.83608983 2.83321976 2.76955352 2.8674714 ]

 
folds:  9
{'xgb_regressor__subsample': 0.6, 'xgb_regressor__reg_lambda': 10, 'xgb_regressor__reg_alpha': 0.01, 'xgb_regressor__random_state': 42, 'xgb_regressor__objective': 'reg:squarederror', 'xgb_regressor__n_estimators': 200, 'xgb_regressor__max_depth': 30, 'xgb_regressor__learning_rate': 0.3, 'xgb_regressor__gamma': 0.5, 'xgb_regressor__colsample_bytree': 0.6, 'tfidf__use_idf': False, 'tfidf__sublinear_tf': True, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 1), 'tfidf__min_df': 5, 'tfidf__max_features': 1000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word'}
root_mean_squared_error:  3.025883362449136
 
mean_test_score:  [3.00047719 3.02588336 2.81219351 2.76970398 2.84235971 2.9217593
 2.86698834 3.00350766 2.80968068 2.8480495 ]



 #CON SVD

folds:  5
{'xgb_regressor__subsample': 1, 'xgb_regressor__reg_lambda': 0, 'xgb_regressor__reg_alpha': 0.1, 'xgb_regressor__random_state': 42, 'xgb_regressor__objective': 'reg:squarederror', 'xgb_regressor__n_estimators': 100, 'xgb_regressor__max_depth': 20, 'xgb_regressor__learning_rate': 0.05, 'xgb_regressor__gamma': 0, 'xgb_regressor__colsample_bytree': 1, 'tfidf__use_idf': False, 'tfidf__sublinear_tf': True, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 1), 'tfidf__min_df': 5, 'tfidf__max_features': 2000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word', 'svd__random_state': 42, 'svd__n_components': 100}
root_mean_squared_error:  3.4395795757043617
 
mean_test_score:  [2.83122156 2.96515367 3.09699659 2.86170339 2.88602632 3.43957958
 3.04399317 2.88464823 2.85373039 2.90546938]


folds:  6
{'xgb_regressor__subsample': 1, 'xgb_regressor__reg_lambda': 0.1, 'xgb_regressor__reg_alpha': 0.1, 'xgb_regressor__random_state': 42, 'xgb_regressor__objective': 'reg:squarederror', 'xgb_regressor__n_estimators': 500, 'xgb_regressor__max_depth': 20, 'xgb_regressor__learning_rate': 0.05, 'xgb_regressor__gamma': 0.4, 'xgb_regressor__colsample_bytree': 1, 'tfidf__use_idf': False, 'tfidf__sublinear_tf': False, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 2, 'tfidf__max_features': 2000, 'tfidf__max_df': 0.75, 'tfidf__analyzer': 'word', 'svd__random_state': 42, 'svd__n_components': 300}
root_mean_squared_error:  3.1174904016280647
 
mean_test_score:  [2.90251648 2.98732675 2.94979991 2.91589344 2.89705201 3.1105241
 3.1174904  2.97058267 2.8984135  2.94111667]


folds:  7
{'xgb_regressor__subsample': 0.8, 'xgb_regressor__reg_lambda': 0.01, 'xgb_regressor__reg_alpha': 1, 'xgb_regressor__random_state': 42, 'xgb_regressor__objective': 'reg:squarederror', 'xgb_regressor__n_estimators': 500, 'xgb_regressor__max_depth': 10, 'xgb_regressor__learning_rate': 0.3, 'xgb_regressor__gamma': 0.3, 'xgb_regressor__colsample_bytree': 0.7, 'tfidf__use_idf': True, 'tfidf__sublinear_tf': True, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 1), 'tfidf__min_df': 2, 'tfidf__max_features': 2000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word', 'svd__random_state': 42, 'svd__n_components': 400}
root_mean_squared_error:  3.072295950916678
 
mean_test_score:  [2.8882336  2.99625175 2.96012651 2.88654206 2.82097511 2.87765466
 2.86609657 3.07229595 2.86111338 2.89542429]


folds:  8
{'xgb_regressor__subsample': 0.9, 'xgb_regressor__reg_lambda': 1, 'xgb_regressor__reg_alpha': 1, 'xgb_regressor__random_state': 42, 'xgb_regressor__objective': 'reg:squarederror', 'xgb_regressor__n_estimators': 500, 'xgb_regressor__max_depth': 20, 'xgb_regressor__learning_rate': 0.3, 'xgb_regressor__gamma': 0.5, 'xgb_regressor__colsample_bytree': 1, 'tfidf__use_idf': True, 'tfidf__sublinear_tf': False, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 1), 'tfidf__min_df': 2, 'tfidf__max_features': 1000, 'tfidf__max_df': 0.75, 'tfidf__analyzer': 'word', 'svd__random_state': 42, 'svd__n_components': 300}
root_mean_squared_error:  3.0462914103801038
 
mean_test_score:  [3.00735441 2.89548136 3.04629141 2.89458644 2.85132475 2.80295479
 2.8394033  2.8548867  2.9969562  2.77377019]


folds:  9
{'xgb_regressor__subsample': 0.7, 'xgb_regressor__reg_lambda': 0.1, 'xgb_regressor__reg_alpha': 0, 'xgb_regressor__random_state': 42, 'xgb_regressor__objective': 'reg:squarederror', 'xgb_regressor__n_estimators': 100, 'xgb_regressor__max_depth': 10, 'xgb_regressor__learning_rate': 0.3, 'xgb_regressor__gamma': 0.4, 'xgb_regressor__colsample_bytree': 0.7, 'tfidf__use_idf': True, 'tfidf__sublinear_tf': True, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 1), 'tfidf__min_df': 5, 'tfidf__max_features': 2000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word', 'svd__random_state': 42, 'svd__n_components': 100}
root_mean_squared_error:  3.1054041166941264
 
mean_test_score:  [2.89459705 2.82756804 3.02581797 2.8293994  2.98901009 2.85505087
 2.82793389 3.10540412 2.89859089 2.87424321]


folds:  10
{'xgb_regressor__subsample': 0.6, 'xgb_regressor__reg_lambda': 1, 'xgb_regressor__reg_alpha': 0.1, 'xgb_regressor__random_state': 42, 'xgb_regressor__objective': 'reg:squarederror', 'xgb_regressor__n_estimators': 500, 'xgb_regressor__max_depth': 20, 'xgb_regressor__learning_rate': 0.3, 'xgb_regressor__gamma': 0, 'xgb_regressor__colsample_bytree': 0.6, 'tfidf__use_idf': False, 'tfidf__sublinear_tf': True, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 1), 'tfidf__min_df': 2, 'tfidf__max_features': 1000, 'tfidf__max_df': 0.75, 'tfidf__analyzer': 'word', 'svd__random_state': 42, 'svd__n_components': 400}
root_mean_squared_error:  3.1751849588891394
 
mean_test_score:  [2.83595358 2.93062112 2.8248009  2.88451685 2.86182348 3.17518496
 3.10279488 3.12063031 2.89006735 2.89186031]


In [ ]:
randomcv_best_score.best_params_

In [ ]:
randomcv_best_score.best_score_

In [ ]:
randomcv_best_score.cv_results_['mean_test_score']

THE BEST

folds: 5 {'xgb_regressor__subsample': 0.7, 'xgb_regressor__reg_lambda': 0.1, 'xgb_regressor__reg_alpha': 0, 'xgb_regressor__random_state': 42, 'xgb_regressor__objective': 'reg:squarederror', 'xgb_regressor__n_estimators': 500, 'xgb_regressor__max_depth': 10, 'xgb_regressor__learning_rate': 0.3, 'xgb_regressor__gamma': 0.1, 'xgb_regressor__colsample_bytree': 1, 'tfidf__use_idf': False, 'tfidf__sublinear_tf': True, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 1), 'tfidf__min_df': 5, 'tfidf__max_features': 2000, 'tfidf__max_df': 0.8, 'tfidf__analyzer': 'word'} root_mean_squared_error: 3.0159159380511493

#CON SVD

folds: 8 {'xgb_regressor__subsample': 0.9, 'xgb_regressor__reg_lambda': 1, 'xgb_regressor__reg_alpha': 1, 'xgb_regressor__random_state': 42, 'xgb_regressor__objective': 'reg:squarederror', 'xgb_regressor__n_estimators': 500, 'xgb_regressor__max_depth': 20, 'xgb_regressor__learning_rate': 0.3, 'xgb_regressor__gamma': 0.5, 'xgb_regressor__colsample_bytree': 1, 'tfidf__use_idf': True, 'tfidf__sublinear_tf': False, 'tfidf__stop_words': ['english'], 'tfidf__ngram_range': (1, 1), 'tfidf__min_df': 2, 'tfidf__max_features': 1000, 'tfidf__max_df': 0.75, 'tfidf__analyzer': 'word', 'svd__random_state': 42, 'svd__n_components': 300} root_mean_squared_error: 3.0462914103801038